In [9]:
""" use kernel tensorflow_p36 - NOT TF 2"""


' use kernel tensorflow_p36 - NOT TF 2'

In [16]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf

from sklearn.model_selection import train_test_split

EPOCHS = 20
IMG_WIDTH = 224
IMG_HEIGHT = 224
NUM_CATEGORIES = 2
TEST_SIZE = 0.2

In [17]:
def load_data(data_dir):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    images = []
    labels=[]
    counter = 0
    for labelname in os.listdir(data_dir):
        path= data_dir + "/" + labelname 
        for pic in os.listdir(path):
            path2= data_dir + "/" + labelname + "/" + pic
            img= cv2.imread(path2)
            #print ("path- ", path)
            #print ("path2- ", path2)
            re_img=cv2.resize(img,(IMG_WIDTH,IMG_HEIGHT))
            images.append(re_img)
            labels.append(labelname)
            counter= counter+1
    print(counter) 
    return (images,labels)


def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
        """
# Create a convolutional neural network
    model = tf.keras.models.Sequential([

         # Convolutional layer. Learn 32 filters using a 3x3 kernel
         tf.keras.layers.Conv2D(
             32, (3, 3), activation="relu", padding = "same", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
         ),


         # Max-pooling layer, using 3x3 pool size
         tf.keras.layers.MaxPooling2D(pool_size=(4, 4)),
         tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Conv2D(
             64, (3, 3), activation="relu", padding = "same", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
         ),


         # Max-pooling layer, using 3x3 pool size
         tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),
         tf.keras.layers.Dropout(0.2),
         

         # Flatten units
         tf.keras.layers.Flatten(),
         tf.keras.layers.Dropout(0.2),
        
        

         # Add a hidden layer with dropout
         tf.keras.layers.Dense(NUM_CATEGORIES*32, activation="relu"),
         tf.keras.layers.Dropout(0.2),
        
         
         # Flatten units
         tf.keras.layers.Flatten(),
         tf.keras.layers.Dropout(0.2),
        
        
        # Add a hidden layer with dropout
         tf.keras.layers.Dense(NUM_CATEGORIES*16, activation="relu"),
         tf.keras.layers.Dropout(0.2),
        

        # Add a hidden layer with dropout
         tf.keras.layers.Dense(NUM_CATEGORIES*8, activation="relu"),
         tf.keras.layers.Dropout(0.2),
         
         # Add an output layer with number of categories
         tf.keras.layers.Dense(NUM_CATEGORIES, activation="softmax")
     ])

# Train neural network
    model.compile(
         optimizer="adam",
         loss="categorical_crossentropy",
         metrics=["accuracy"]
   )

    return model



In [18]:
# Get image arrays and labels for all image files
    images, labels = load_data("/home/ec2-user/SageMaker/CNN/brain")

    labels = tf.keras.utils.to_categorical(labels)
    x_train, x_test, y_train, y_test = train_test_split(
        np.array(images), np.array(labels), test_size=TEST_SIZE)


253


In [19]:
model = get_model()


In [20]:
import time

start = time.time()

# Fit model on training data
model_history = model.fit(x_train, y_train, epochs=EPOCHS)

end = time.time()
print(end - start)

Train on 202 samples
Epoch 1/20
202/202 [==============================] - 2s 10ms/sample - loss: 30.3425 - acc: 0.5644
Epoch 2/20
202/202 [==============================] - 1s 6ms/sample - loss: 18.8523 - acc: 0.5792
Epoch 3/20
202/202 [==============================] - 1s 6ms/sample - loss: 1.4941 - acc: 0.6535
Epoch 4/20
202/202 [==============================] - 1s 6ms/sample - loss: 0.8209 - acc: 0.6881
Epoch 5/20
202/202 [==============================] - 1s 6ms/sample - loss: 0.7975 - acc: 0.6436
Epoch 6/20
202/202 [==============================] - 1s 6ms/sample - loss: 0.5793 - acc: 0.7624
Epoch 7/20
202/202 [==============================] - 1s 6ms/sample - loss: 0.6727 - acc: 0.6634
Epoch 8/20
202/202 [==============================] - 1s 6ms/sample - loss: 0.5394 - acc: 0.7525
Epoch 9/20
202/202 [==============================] - 1s 6ms/sample - loss: 0.5967 - acc: 0.7228
Epoch 10/20
202/202 [==============================] - 1s 6ms/sample - loss: 0.5203 - acc: 0.7624
Epoch

In [21]:
   # Evaluate neural network performance
    model.evaluate(x_test,  y_test, verbose=2)

51/51 - 0s - loss: 0.5692 - acc: 0.6275


[0.5691947224093419, 0.627451]

In [22]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 18, 18, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 20736)            

In [23]:
model.save("model-clean")